# Problemy wydawania reszty i problem plecakowy

W tej lekcji zajmiemy się dwoma problemami algorytmicznymi. Oba one posiadają bardzo proste sformułowania - nie dajmy się jednak nabrać - gdyż okazuje się, że oba należą do grupy problemów NP. Przyjrzyjmy się obu zaczynając od nieco prostszego z nich.

# Problem wydawania reszty

Problem wydawania reszty można sformułować następująco:

*Przypuśćmy, że zawieramy transakcję z pewną osobą. W jej efekcie musimy wydać pewną ilość pieniędzy. Otwieramy swój portfel i musimy teraz wybrać konkretne nominały aby 'zbudować' odliczoną kwotę*

## Definicja

Niech będzie dany zbiór nominałów $n_1, n_2, \ldots, n_n$ (posortowanych rosnąco w sposób ostry $i < j \Rightarrow n_i < n_j$) i kwota do wydania $C$. Wyznaczamy takie współczynniki nieujemne całkowite $x_1,\ldots, x_n$, że 
$$\sum_{j=1}^n n_j x_j = C,$$
oraz użyto najmniejszej liczby monet tj. wartość
$$\sum_{j=1}^n x_j$$
jest najmniejsza.

## Opis problemu

Jak widzimy problem oprócz samego wydania reszty zakłada minimalizację formy tej reszty. U jego podstaw leży oczywiście jakaś forma grzeczności - wydawanie zbyt dużej ilości monet/banknotów (tzw. rozliczanie się 1-groszówkami) wydaje się nieeleganckie i kłopotliwe dla drugiej strony. 

Problem ten występuje w dwóch wersjach:
* klasycznej z nieograniczonym zasobem każdego z nominałów - w tej wersji jest on rozważany najczęściej przez algorytmików, oraz
* z dodatkowymi ograniczeniami - w tej wersji dla każdego z nominałów może być dostępna jedynie określona ilość monet. Mamy więc dodatkowe ograniczenia postaci $x_i \leq b_i$. 

Dla problemu kluczowa okazuje się jednak postać zbioru z nominałami. Istnieją matematyczne wyniki orzekające, że niektóre takie systemy nominałów posiadają szybkie liniowe algorytmy rozwiązujące. Istnieją również systemy nominałów przy których algorytmy te zawodzą a poszukiwanie rozwiązania problemu zdaje się mieć wykładniczą złożoność.

Przykładem 'dobrego układu nominałów' może być ten którym się posługujemy tj. 1,2,5,10,20,50,100,200,500.
Dla takich zbiorów nominałów optymalny okazuje się być następujący algorytm.

## Algorytm zachłanny dla problemu reszty

Podstawowym algorytmem rozwiązywania problemu jest tzw. algorytm zachłanny - czyli algorytm gdzie w danej sytuacji podejmujemy chwilowo najlepszą decyzję. Algorytmy te opierają się na przypuszczeniu, że rozwiązanie optymalne powstaje poprzez podejmowanie najlepszych decyzji na każdym etapie poszukiwań. Jak łatwo przypuszczać 

* algorytm taki, generuje jakościowo dobre rozwiązania problemów, ale
* nie zawsze uzyskuje on jakościowo najlepsze rozwiązanie.

W przypadku problemu wydawania reszty algorytm zachłanny polega na wybieraniu największego dostępnego nominału (tak aby maksymalnie pomniejszyć kwotę do dalszej wypłaty). Przyjrzyjmy się jego prostej implementacji

In [1]:
K = 132

nominaly = [1, 2, 5, 10, 20, 50]

In [2]:
def change_making_greedy(K, nominaly):
    nominaly.sort(reverse = True)
    coins = []
    for i in nominaly:
        while K >= i:
            coins.append(i)
            K -= i
    return coins

In [3]:
change_making_greedy(K, nominaly)

[50, 50, 20, 10, 2]

Jeśli interesuje nas ile monet potrzeba aby wypłacić resztę wystarczyło by dodać


In [4]:
len(change_making_greedy(K, nominaly))

5

Zauważmy jednak, że algorytm ten potrafi znaleźć nieoptymalne rozwiązanie. Rozważmy przykład


In [5]:
K = 8
nominaly = [1,4,5]
change_making_greedy(K,nominaly)

[5, 1, 1, 1]

Tymczasem

In [6]:
4+4

8

## Algorytm z zastosowaniem programowania dynamicznego

Ogólna technika odszukiwania optymalnego rozwiązania wymaga bardziej zaawansowanego algorytmu. Dobre wyniki są zwracane przez techniki programowania dynamicznego. Idea jest następująca:

1. Poszukujemy optymalnego sposobu na wydanie reszt mniejszych kwot,
1. Budujemy reszty wyższych kwot poprzez parowanie wydania mniejszych reszt.

Jak to działa. Wyobraźmy sobie, że mamy do wydania kwotę 100. 
Kwotę 100 można np. podzielić na 1+99. Jeśli mamy, że 99 możemy optymalnie wydać za pomocą k monet a 1 - 1 jedną monetą - to mamy pomysł by wydać 100 za pomocą k+1 monet. Nie możemy jednak na tym poprzestać. Trzeba sprawdzić 

* 2 + 98, 
* 3 + 97,
* 4 + 96,
* ...
* 50 + 50;

I ze wszystkich tych możliwości wybrać tę, która generuje resztę o minimalnej ilości monet.

In [7]:
def change_making_dynamic(K, nominaly, maks_size = 1000000):
    change_making = {}
    for i in range(1,K+1):
        change_making[i] = [] #najpierw nie znamy przepisu na zadna z liczb
    for i in range(1,K+1):
        if i in nominaly: #jesli nasza liczba jest wsrod dostepnych nominalow - to najprosciej wydac ja na raz
            change_making[i].append(i)
            continue # nie ma bardziej optymalnego sposobu
        # inaczej rozwazamy wszystkie pary i wyszukujemy najmniejszego zestawu reszt
        min_size = maks_size
        min_ind = 1
        for j in range(1,i):
            # len(change_making[j]) + len(change_making[i-j]) to ilosc monet w reszcie przy probie podzialu
            # i na j + (i-j)
            if min_size > len(change_making[j]) + len(change_making[i-j]):
                min_size = len(change_making[j]) + len(change_making[i-j])
                min_ind = j
        # przypisanie najoptymalniejszego podziału (dowolnego tj pierwszego) do tablicy wyników
        change_making[i] = change_making[min_ind].copy()
        change_making[i].extend(change_making[i-min_ind])
    return change_making

In [8]:
K = 20
nominaly = [1,4,5]
change_making_dynamic(K,nominaly)

{1: [1],
 2: [1, 1],
 3: [1, 1, 1],
 4: [4],
 5: [5],
 6: [1, 5],
 7: [1, 1, 5],
 8: [4, 4],
 9: [4, 5],
 10: [5, 5],
 11: [1, 5, 5],
 12: [4, 4, 4],
 13: [4, 4, 5],
 14: [4, 5, 5],
 15: [5, 5, 5],
 16: [1, 5, 5, 5],
 17: [4, 4, 4, 5],
 18: [4, 4, 5, 5],
 19: [4, 5, 5, 5],
 20: [5, 5, 5, 5]}

# Problem plecakowy

Problem plecakowy jest problemem nieco ogólniejszym od problemu wydawania reszty. Formułuje się go następująco:

*Przypuśćmy, że ruszamy w podróż pieszą i możemy zabrać tylko pewną określoną ilość rzeczy. Z reguły ograniczeniem dla nas jest masa - gdyż nie możemy podróżować ze zbyt ciężkim plecakiem. Chcemy jednak aby plecak zawierał jedynie najpotrzebniejsze/najcenniejsze dla nas przedmioty.*

## Definicja 

Rozważmy zbiór $n$ przedmiotów gdzie dla $i$-tego przedmiotu $v_i$ oznaczać będzie jego wartość oraz $w_i$ jego wagę. Poszukujemy takiego  układu wartości $x_i$, że dla każdego $i \in \{1,\ldots, n\}$ zachodzi $x_i \in \{ 0, 1 \}$ spełniającego warunek nieprzepełniania plecaka
$$
\sum_{i=1}^{n} w_i \cdot x_i \leq W,
$$
przy maksymalizacji jego wartości
$$
\sum_{i=1}^{n} v_i \cdot x_i \to \max. 
$$

## Algorytm zachłanny

Dla algorytmu plecakowego możemy rozważyć algorytm przybliżonego rozwiązywanie problemu w sposób zachłanny. Jego postać jest następująca

1. Sortujemy obiekty wg malejącego stosunku cena do wagi.
1. W danej chwili wybieramy pierwszy z listy element, który mieści się do plecaka



In [9]:
# ['nazwa przedmiotu', wartosc, waga ]
items = [ ['a', 2, 4] , ['b', 4, 2] , ['c', 2, 2], ['d', 3, 3], ['e', 4, 1] ]

size = 6

In [10]:
import copy

def knapsack_greedy(items, size):
    inner_items = copy.deepcopy(items)
    for item in inner_items:
        item.append(item[1]/item[2])
    inner_items.sort(key=lambda x : x[3], reverse = True)
    knapsack = {}
    for item in inner_items:
        if item[2] <= size:
            size -= item[2]
            knapsack[item[0]] = item[:]
    return knapsack

In [11]:
knapsack_greedy(items, size)

{'e': ['e', 4, 1, 4.0], 'b': ['b', 4, 2, 2.0], 'c': ['c', 2, 2, 1.0]}

## Algorytm przeglądu zupełnego

W przypadku gdy przedmioty mają nieskończoną liczność - możliwy jest analogiczny do problemu reszty - algorytm programowania dynamicznego. Jednak w tym najczęściej spotykanym przypadku - jedyny algorytm odszukujący poprawne rozwiązanie problemu jest algorytm przeglądu całkowitego.



In [12]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

for i in powerset(items):
    print(i)

()
(['a', 2, 4],)
(['b', 4, 2],)
(['c', 2, 2],)
(['d', 3, 3],)
(['e', 4, 1],)
(['a', 2, 4], ['b', 4, 2])
(['a', 2, 4], ['c', 2, 2])
(['a', 2, 4], ['d', 3, 3])
(['a', 2, 4], ['e', 4, 1])
(['b', 4, 2], ['c', 2, 2])
(['b', 4, 2], ['d', 3, 3])
(['b', 4, 2], ['e', 4, 1])
(['c', 2, 2], ['d', 3, 3])
(['c', 2, 2], ['e', 4, 1])
(['d', 3, 3], ['e', 4, 1])
(['a', 2, 4], ['b', 4, 2], ['c', 2, 2])
(['a', 2, 4], ['b', 4, 2], ['d', 3, 3])
(['a', 2, 4], ['b', 4, 2], ['e', 4, 1])
(['a', 2, 4], ['c', 2, 2], ['d', 3, 3])
(['a', 2, 4], ['c', 2, 2], ['e', 4, 1])
(['a', 2, 4], ['d', 3, 3], ['e', 4, 1])
(['b', 4, 2], ['c', 2, 2], ['d', 3, 3])
(['b', 4, 2], ['c', 2, 2], ['e', 4, 1])
(['b', 4, 2], ['d', 3, 3], ['e', 4, 1])
(['c', 2, 2], ['d', 3, 3], ['e', 4, 1])
(['a', 2, 4], ['b', 4, 2], ['c', 2, 2], ['d', 3, 3])
(['a', 2, 4], ['b', 4, 2], ['c', 2, 2], ['e', 4, 1])
(['a', 2, 4], ['b', 4, 2], ['d', 3, 3], ['e', 4, 1])
(['a', 2, 4], ['c', 2, 2], ['d', 3, 3], ['e', 4, 1])
(['b', 4, 2], ['c', 2, 2], ['d', 3, 3], 

In [13]:
def knapsack_full(items, size):
    max_combination = None
    max_value = 0
    
    def value_and_weight(combination):
        value = 0
        weight = 0
        for item in combination:
            value += item[1]
            weight += item[2]
        return value, weight
    
    for combination in powerset(items):
        value, weight = value_and_weight(combination)
        if weight > size:
            continue
        if value > max_value:
            max_value = value
            max_combination = combination
    return max_combination

In [14]:
knapsack_full(items, size)

(['b', 4, 2], ['d', 3, 3], ['e', 4, 1])